## Movies data

### Explore and clean the data
---

In [1]:
####  Import statements
import pandas as pd

df = pd.read_csv("cost_revenue_dirty.csv")
df.sample(5)    # Checking a random sample of 5

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
2603,989,2/6/2006,The Break-Up,"$52,000,000","$205,727,307","$118,703,275"
2382,2837,2/18/2005,Because of Winn-Dixie,"$15,000,000","$33,508,485","$32,647,042"
1863,3147,8/21/2002,One Hour Photo,"$12,000,000","$52,223,306","$31,597,131"
1810,274,5/16/2002,Star Wars Ep. II: Attack of the Clones,"$115,000,000","$656,695,615","$310,676,740"
457,3829,12/19/1986,Platoon,"$6,000,000","$137,963,328","$137,963,328"


#### Initial checks

In [2]:
#### Initial checks
df.head()
df.shape
df.columns      # Checks are fine
df.dtypes       # Last three usd columns need to be converted.

#Adjustments

    # Convert the currency to numbers
columns = ["USD_Production_Budget", "USD_Worldwide_Gross", "USD_Domestic_Gross"]
for column in columns:
    df[column]= df[column].astype(str).str.replace("$", "")
    df[column]= df[column].astype(str).str.replace(",", "")
    df[column] =pd.to_numeric(df[column])

    # convert the date to datetime
df["Release_Date"] = pd.to_datetime(df["Release_Date"])
df.dtypes


Rank                              int64
Release_Date             datetime64[ns]
Movie_Title                      object
USD_Production_Budget             int64
USD_Worldwide_Gross               int64
USD_Domestic_Gross                int64
dtype: object

In [3]:
# Secondary checks
    # check for missing data
df.isna().sum()
        # No missing data so no changes needed

    # checks for duplicates
        # Helps you to see them ( keep= false mean to keep all, can do first and the last.)
df [df[["Movie_Title","Release_Date"]].duplicated(keep = False)].sort_values("Movie_Title")
            # Intially did moveit title but the release dates are different.
            ## When accounting for release date and movie name there is only one duplicates
df[["Movie_Title", "Release_Date"]].duplicated().value_counts()
            # Only one item that identical

df.drop_duplicates(subset =["Movie_Title", "Release_Date"], keep ="first", inplace = True)
df[["Movie_Title", "Release_Date"]].duplicated().value_counts()


False    5390
Name: count, dtype: int64

#### Descriptive stats

##### Challenge 1
What is the average production budget of the films in the data set?

What is the average worldwide gross revenue of films?

What were the minimums for worldwide and domestic revenue?

Are the bottom 25% of films actually profitable or do they lose money?

What are the highest production budget and highest worldwide gross revenue of any film?

How much revenue did the lowest and highest budget films make?

In [4]:
df.head()
pd.set_option('display.float_format', '{:,.2f}'.format)
df.describe()

,Rank,Release_Date,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
count,"5,390.00",5390,"5,390.00","5,390.00","5,390.00"
mean,"2,695.52",2003-09-19 08:04:21.818181888,"31,119,487.81","88,871,907.20","41,243,169.82"
min,1.00,1915-08-02 00:00:00,"1,100.00",0.00,0.00
25%,"1,348.25",1999-12-02 06:00:00,"5,000,000.00","3,877,146.00","1,335,065.75"
50%,"2,695.50",2006-06-23 00:00:00,"17,000,000.00","27,460,037.00","17,196,565.00"
75%,"4,042.75",2011-11-23 00:00:00,"40,000,000.00","96,461,821.00","52,348,661.50"
max,"5,391.00",2020-12-31 00:00:00,"425,000,000.00","2,783,918,982.00","936,662,225.00"
std,"1,556.14",NaN,"40,525,356.93","168,469,037.10","66,033,082.74"


### Analysing data
---


#### Boolean indexing ---- way to make multiple comparisons

In [10]:
### Use loc and boolean indexing to work out the ones where domestic ==0 and worldwide !=0
international_releases =  df.loc[(df["USD_Domestic_Gross"]== 0) & (df["USD_Worldwide_Gross"]!=0)]
international_releases

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
71,4310,1956-02-16,Carousel,3380000,3220,0
1579,5087,2001-02-11,Everything Put Together,500000,7890,0
1744,3695,2001-12-31,The Hole,7500000,10834406,0
2155,4236,2003-12-31,Nothing,4000000,63180,0
2203,2513,2004-03-31,The Touch,20000000,5918742,0
...,...,...,...,...,...,...
5340,1506,2017-04-14,Queen of the Desert,36000000,1480089,0
5348,2225,2017-05-05,Chāi dàn zhuānjiā,23000000,58807172,0
5360,4832,2017-07-03,Departure,1100000,27561,0
5372,1856,2017-08-25,Ballerina,30000000,48048527,0


#### Using Query -- filter by multiple conditions. ( Same thing as before but just using strings)

In [11]:
international_releases2 =  df.query("USD_Domestic_Gross==0  and USD_Worldwide_Gross!=0")
international_releases2.head()

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
71,4310,1956-02-16,Carousel,3380000,3220,0
1579,5087,2001-02-11,Everything Put Together,500000,7890,0
1744,3695,2001-12-31,The Hole,7500000,10834406,0
2155,4236,2003-12-31,Nothing,4000000,63180,0
2203,2513,2004-03-31,The Touch,20000000,5918742,0


####  Challenge
Now we can turn our attention to films in the dataset that were not released at the time the data was collected. This is why films like Singularity and Aquaman had zero revenue.



Identify which films were not released yet as of the time of data collection (May 1st, 2018).

How many films are included in the dataset that have not yet had a chance to be screened in the box office?

Create another DataFrame called data_clean that does not include these films.

In [13]:
#### Filter by the date
scrape_date = pd.Timestamp("2018-05-01")                        # Convert the date you look for into datetime
future_release = df[df["Release_Date"] > scrape_date]           # make the comparison to the dates
future_release.head()


## Removing these from the list
ddata_clean  = df.drop(future_release.index)                    # Drop the values from their index values.